In [5]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

import os

## Ignore warnings
import warnings
def warn(*args, **kwargs):
    pass
warnings.warn = warn
np.warnings.filterwarnings('ignore')

os.chdir('/tld')
from maps.maps import fine_linktype_map
from maps.maps import status_map
from maps.maps import resolution_map
os.chdir('..')
os.getcwd()

'/informatik2/mobis/home/lueders/Desktop/Forschung/Research/DISS/SotP'

In [6]:
SOURCES = ['Apache', 'Hyperledger', 'IntelDAOS', 'JFrog', 'Jira', 
           'JiraEcosystem', 'MariaDB', 'Mindville', 'Mojang', 'MongoDB', 
           'Qt', 'RedHat', 'Sakai', 'SecondLife', 'Sonatype', 'Spring']

In [7]:
def load_cleaned_links(source):
    #Loading Issues
    filename = '../data/cleaned_links_'+source.lower()+'.csv'
    link_df = pd.read_csv(filename, encoding="UTF-8", low_memory=False, sep=';')
    link_df['maptype'] = link_df['linktype'].map(fine_linktype_map)
    
    return link_df

def load_uncleaned_links(source):
    #Loading Issues
    filename = '../data/links_'+source.lower()+'.csv'
    link_df = pd.read_csv(filename, encoding="UTF-8", low_memory=False, sep=';')
    link_df['maptype'] = link_df['linktype'].map(fine_linktype_map)
    
    return link_df

def load_detailed_issues(source):
    #Loading Issues
    filename = '../data/detailed_issues_'+source.lower()+'.csv'
    issue_df = pd.read_csv(filename, encoding="UTF-8", low_memory=False, sep=',')
    
    return issue_df



In [53]:
temp = []
for s in SOURCES:
    issues = issue_dict[s]
    temp.append(len(issues[issues['description'].isna()])/len(issues))
    print(round((len(issues[issues['description'].isna()])/len(issues))*100,2))
temp.mean(bla)

8.35
16.15
11.56
9.17
3.79
19.51
3.7
0.89
1.63
12.11
3.86
12.7
0.99
0.0
21.25
14.64


0.08767711675900643

In [8]:
un_link_dict = {}

for s in SOURCES:
    print(s.upper())
    
    links = load_uncleaned_links(s)
    un_link_dict[s] = links

APACHE
HYPERLEDGER
INTELDAOS
JFROG
JIRA
JIRAECOSYSTEM
MARIADB
MINDVILLE
MOJANG
MONGODB
QT
REDHAT
SAKAI
SECONDLIFE
SONATYPE
SPRING


In [9]:
link_dict = {}
issue_dict = {}

for s in SOURCES:
    print(s.upper())
    
    links = load_cleaned_links(s)
    link_dict[s] = links
    
    issues = load_detailed_issues(s)
    issue_dict[s] = issues

APACHE
HYPERLEDGER
INTELDAOS
JFROG
JIRA
JIRAECOSYSTEM
MARIADB
MINDVILLE
MOJANG
MONGODB
QT
REDHAT
SAKAI
SECONDLIFE
SONATYPE
SPRING


## Issue Type

In [10]:
def check_issue_type(type_1, type_2, type_to_check):
    return (type_1==type_to_check)^(type_2==type_to_check)

In [11]:
esl = []
ecl = []

for s in SOURCES:
    print(s)
    issues = issue_dict[s]
    links = link_dict[s]
            
    projects = issues[['issue_id', 'type']]

    temp = links.merge(projects, left_on = 'issue_id_1', right_on='issue_id')
    temp = temp.merge(projects, left_on = 'issue_id_2', right_on='issue_id', suffixes=('_1', '_2'))

    epic_links = temp[temp.maptype == 'Epic']
    correct = []
    for i in epic_links.index:
        issue_type_1 = epic_links['type_1'].loc[i]
        issue_type_2 = epic_links['type_2'].loc[i]
        correct.append(check_issue_type(issue_type_1, issue_type_2, 'Epic'))
        
    epic_links['correct_link'] = correct
    
    if len(epic_links)!=0:
        epi_share = len(epic_links[epic_links['correct_link']==False])/len(epic_links)
        epi_count = len(epic_links[epic_links['correct_link']==False])
        print(len(epic_links[epic_links['correct_link']==False])/len(epic_links))
        print(epic_links[epic_links['correct_link']==False])
        esl.append(epi_share)
        ecl.append(epi_count)
    

Apache
0.0
Empty DataFrame
Columns: [Unnamed: 0, id, linktype, issue_id_1, issue_id_2, issues, maptype, issue_id_1, type_1, issue_id_2, type_2, correct_link]
Index: []
Hyperledger
0.0
Empty DataFrame
Columns: [Unnamed: 0, id, linktype, issue_id_1, issue_id_2, issues, maptype, issue_id_1, type_1, issue_id_2, type_2, correct_link]
Index: []
IntelDAOS
JFrog
Jira
JiraEcosystem
0.000723589001447178
      Unnamed: 0           id linktype issue_id_1 issue_id_2  \
308          425   30806_Epic     Epic   UPM-3616  AMKT-5004   
6020        8991  151620_Epic     Epic   AUI-5013   AUI-5066   

                         issues maptype issue_id_1 type_1 issue_id_2  \
308   ['AMKT-5004', 'UPM-3616']    Epic   UPM-3616   Epic  AMKT-5004   
6020   ['AUI-5013', 'AUI-5066']    Epic   AUI-5013    Bug   AUI-5066   

             type_2  correct_link  
308            Epic         False  
6020  Documentation         False  
MariaDB
0.0
Empty DataFrame
Columns: [Unnamed: 0, id, linktype, issue_id_1, issue_id_

In [13]:
print(np.mean(esl))
print(np.std(esl))

0.00010275540867996193
0.0002368556769948154


In [14]:
esi = []
eci = []

for s in SOURCES:
    print(s)
    issues = issue_dict[s]
    links = un_link_dict[s]

    epic_issues = set(issues[issues.type == 'Epic']['issue_id'])
    linked_issues = set(links['issue_id_1']).union(set(links['issue_id_2']))
    
    print(len(epic_issues - linked_issues))
    if(len(epic_issues)!=0):
        epi_share = len(epic_issues - linked_issues)/len(epic_issues)
        epi_count = len(epic_issues - linked_issues)
        print(len(epic_issues - linked_issues)/len(epic_issues))
        print(issues[issues['issue_id'].isin(list(epic_issues - linked_issues)[:10])][['issue_id', 'type', 'no_links']])
        esi.append(epi_share)
        eci.append(epi_count)

Apache
160
0.13136288998357964
             issue_id  type  no_links
114739   SYSTEMDS-715  Epic         0
114763   SYSTEMDS-691  Epic         0
377029     MESOS-7787  Epic         0
378265     MESOS-6548  Epic         0
784916    DATALAB-516  Epic         0
966026   AMBARI-23591  Epic         0
997287  AIRAVATA-2722  Epic         0
997292  AIRAVATA-2717  Epic         0
998115  AIRAVATA-1894  Epic         0
999230   AIRAVATA-766  Epic         0
Hyperledger
509
0.41517128874388254
        issue_id  type  no_links
3324     IR-1118  Epic         0
6907      GRID-9  Epic         0
10717  FABCN-219  Epic         0
10720  FABCN-216  Epic         0
10790  FABCN-145  Epic         0
10804  FABCN-131  Epic         0
11064  FABCJ-172  Epic         0
11081  FABCJ-155  Epic         0
11107  FABCJ-129  Epic         0
11124  FABCJ-112  Epic         0
IntelDAOS
241
0.8576512455516014
       issue_id  type  no_links
1983  DAOS-7358  Epic         0
2030  DAOS-7308  Epic         0
3892  DAOS-4730  Epic  

In [15]:
print(np.mean(esi))
print(np.std(esi))

0.36709972297230764
0.2608540995230571


In [16]:
def check_issue_type_2(type_1, type_2, type_list):
    exp_1 = type_1 in type_list
    exp_2 = type_2 in type_list
    exp_3 = (type_1 != type_2)
    return (exp_1 or exp_2) and exp_3

In [17]:
subtask_names = ['Sub-task', "Technical task", "Task", 'Sub-Task', 'Backport Sub-Task',
                     'Component Upgrade Subtask', 'Dev Sub-task', 'Sub-requirement', 'QE Sub-task',
                     'Docs Sub-task', 'Simple Sub-task', 'Sub-bug', 'Sub-requirement', 'RCA(sub)', 'Change Request']

In [18]:
ssl = []
scl = []

for s in SOURCES:
    print(s)
    issues = issue_dict[s]
    links = link_dict[s]
            
    projects = issues[['issue_id', 'type']]

    temp = links.merge(projects, left_on = 'issue_id_1', right_on='issue_id')
    temp = temp.merge(projects, left_on = 'issue_id_2', right_on='issue_id', suffixes=('_1', '_2'))
    
    st_links = temp[temp.maptype=='Subtask']
    correct = []
    for i in st_links.index:
        issue_type_1 = st_links['type_1'].loc[i]
        issue_type_2 = st_links['type_2'].loc[i]
        correct.append(check_issue_type_2(issue_type_1, issue_type_2, subtask_names))
        
    st_links['correct_link'] = correct
    
    if(len(st_links)!=0):
        sub_count = len(st_links[st_links['correct_link']==False])
        sub_share = len(st_links[st_links['correct_link']==False])/len(st_links)
        print(len(st_links[st_links['correct_link']==False]))
        print(len(st_links[st_links['correct_link']==False])/len(st_links))
        if(len(st_links[st_links['correct_link']==False])!=0):
            print(st_links[st_links['correct_link']==False])
        ssl.append(sub_share)
        scl.append(sub_count)
    

Apache
320
0.0038208042793007927
        Unnamed: 0                id linktype   issue_id_1   issue_id_2  \
60183        89573  12491311_Subtask  Subtask  SMXCOMP-779  SMX4NMR-212   
60185        89575  12490640_Subtask  Subtask  SMXCOMP-710  SMXCOMP-592   
60186        89576  12490803_Subtask  Subtask  SMXCOMP-677  SMXCOMP-730   
60190        89580  12490447_Subtask  Subtask  SMXCOMP-581  SMXCOMP-618   
60192        89584  12490696_Subtask  Subtask  SMXCOMP-568  SMXCOMP-125   
...            ...               ...      ...          ...          ...   
248958      419362  12481464_Subtask  Subtask      AMQ-286      AMQ-459   
248964      419372  12481470_Subtask  Subtask      AMQ-201       AMQ-60   
248966      419376  12481425_Subtask  Subtask      AMQ-116       AMQ-65   
248969      419381  12481284_Subtask  Subtask       AMQ-27      AMQ-107   
248970      419385  12487588_Subtask  Subtask       AMQ-20    CAMEL-366   

                                issues  maptype   issue_id_1      

0
0.0
Spring
66
0.03400309119010819
      Unnamed: 0             id linktype issue_id_1 issue_id_2  \
2449        3038  47256_Subtask  Subtask   SWF-1577   SWF-1587   
7027       11020  45937_Subtask  Subtask   SPR-9925   SPR-9939   
7070       10403  51427_Subtask  Subtask  SPR-10997  SPR-11011   
7071       10404  51401_Subtask  Subtask  SPR-10996  SPR-11009   
7096       10435  51428_Subtask  Subtask  SPR-10945  SPR-11012   
...          ...            ...      ...        ...        ...   
7657       11619  45735_Subtask  Subtask   SPR-9017   SPR-9879   
7662       11705  46270_Subtask  Subtask   SPR-8896  SPR-10009   
7665       11716  43925_Subtask  Subtask   SPR-8883   SPR-9520   
8015       12374  43694_Subtask  Subtask   SPR-7938   SPR-9460   
8232       12921  43584_Subtask  Subtask   SPR-6870   SPR-9448   

                          issues  maptype issue_id_1       type_1 issue_id_2  \
2449    ['SWF-1577', 'SWF-1587']  Subtask   SWF-1577          Bug   SWF-1587   
7027    ['S

In [19]:
print(np.mean(ssl))
print(np.std(ssl))

0.015518159318211202
0.045986500983829635


In [20]:
subtask_names = ['Sub-task', 'Sub-Task', 'Backport Sub-Task',
                     'Component Upgrade Subtask', 'Dev Sub-task', 'Sub-requirement', 'QE Sub-task',
                     'Docs Sub-task', 'Simple Sub-task', 'Sub-bug', 'Sub-requirement']

In [21]:
ssi = []
sci = []

for s in SOURCES:
    print(s)
    issues = issue_dict[s]
    links = un_link_dict[s]

    st_issues = set(issues[issues.type.isin(subtask_names)]['issue_id'])
    linked_issues = set(links['issue_id_1']).union(set(links['issue_id_2']))
    
    print(len(st_issues))
    print(len(st_issues - linked_issues))
    if(len(st_issues)!=0):
        sub_share = len(st_issues - linked_issues)/len(st_issues)
        sub_count = len(st_issues - linked_issues)
        ssi.append(sub_share)
        sci.append(sub_count)
        print(len(st_issues - linked_issues)/len(st_issues))
        print(issues[issues['issue_id'].isin(list(st_issues - linked_issues)[:10])][['issue_id', 'type', 'no_links']])
    

Apache
83854
382
0.004555537004794046
               issue_id      type  no_links
249922        QPID-4217  Sub-task         0
421589          LIVY-27  Sub-task         0
572198       HDFS-11542  Sub-task         0
787635     DAFFODIL-459  Sub-task         0
787709     DAFFODIL-385  Sub-task         0
788009      DAFFODIL-85  Sub-task         0
788024      DAFFODIL-70  Sub-task         0
952239  APEXMALHAR-1943  Sub-task         0
952241  APEXMALHAR-1941  Sub-task         0
952305  APEXMALHAR-1872  Sub-task         0
Hyperledger
4577
1
0.00021848372296263927
       issue_id      type  no_links
21359  FAB-7651  Sub-task         0
IntelDAOS
274
0
0.0
Empty DataFrame
Columns: [issue_id, type, no_links]
Index: []
JFrog
366
0
0.0
Empty DataFrame
Columns: [issue_id, type, no_links]
Index: []
Jira
2501
0
0.0
Empty DataFrame
Columns: [issue_id, type, no_links]
Index: []
JiraEcosystem
2294
0
0.0
Empty DataFrame
Columns: [issue_id, type, no_links]
Index: []
MariaDB
469
0
0.0
Empty DataFrame
Colum

In [22]:
print(np.mean(ssi))
print(np.std(ssi))

0.0007064138906787438
0.001490969385022044


## Status

In [23]:
#make difference between none and none set / tbd?
stats = ['Closed', 'Open', 'In Progress']

stat_freq_df = pd.DataFrame(columns=stats)

for s in issue_dict.keys():
    print(s)
    temp = []
    issues = issue_dict[s]
    issues['issuestatus'] = issues['status'].map(status_map)
#     print(issues.issuestatus.value_counts())
    if s == "JiraEcosystem":
        issues.loc[issues['status']=='Verified', 'issuestatus'] = 'Open'
        issues.loc[issues['status']=='Reviewing' , 'issuestatus'] = 'Open'
        issues.loc[issues['status']=='Needs Verification' , 'issuestatus'] = 'Closed'
#         print(issues.issuestatus.value_counts())
    if s == "Jira":
        issues.loc[issues['status']=='Reviewing' , 'issuestatus'] = 'Open'
        issues.loc[issues['status']=='Ready for Dev' , 'issuestatus'] = 'Open'
        issues.loc[issues['status']=='Not Being Considered' , 'issuestatus'] = 'Open'
#         print(issues.issuestatus.value_counts())
    if s == "JFrog":
        issues.loc[issues['status']=='Ready for Dev' , 'issuestatus'] = 'Open'
        issues.loc[issues['status']=='Deferred' , 'issuestatus'] = 'Closed'
#         print(issues.issuestatus.value_counts())
    if s == "MongoDB":
        issues.loc[issues['status']=='Designing' , 'issuestatus'] = 'In Progress'
        issues.loc[issues['status']=='Accepted' , 'issuestatus'] = 'Open'
#         print(issues.issuestatus.value_counts())
    if s == "Qt":
        issues.loc[issues['status']=='Blocked' , 'issuestatus'] = 'Open'
#         print(issues.issuestatus.value_counts())
    for p in stats:
        try:
            temp.append(issues.issuestatus.value_counts()[p]/(len(issues)))
        except:
            temp.append(0)
    stat_freq_df.loc[s] = temp

Apache
Hyperledger
IntelDAOS
JFrog
Jira
JiraEcosystem
MariaDB
Mindville
Mojang
MongoDB
Qt
RedHat
Sakai
SecondLife
Sonatype
Spring


In [24]:
print(round(stat_freq_df*100,2).to_latex())

\begin{tabular}{lrrr}
\toprule
{} &  Closed &   Open &  In Progress \\
\midrule
Apache        &   82.31 &  16.50 &         1.19 \\
Hyperledger   &   90.74 &   8.32 &         0.93 \\
IntelDAOS     &   88.67 &   9.24 &         2.09 \\
JFrog         &   66.74 &  32.89 &         0.37 \\
Jira          &   79.31 &  20.25 &         0.44 \\
JiraEcosystem &   89.23 &   9.57 &         1.20 \\
MariaDB       &   74.63 &  20.52 &         4.85 \\
Mindville     &   28.87 &  70.76 &         0.37 \\
Mojang        &   96.60 &   3.40 &         0.00 \\
MongoDB       &   89.21 &   9.87 &         0.93 \\
Qt            &   81.34 &  18.23 &         0.43 \\
RedHat        &   88.75 &   9.90 &         1.35 \\
Sakai         &   88.51 &  10.96 &         0.52 \\
SecondLife    &   83.72 &  10.44 &         5.84 \\
Sonatype      &   98.50 &   1.42 &         0.09 \\
Spring        &   89.02 &  10.11 &         0.87 \\
\bottomrule
\end{tabular}



In [25]:
stat_freq_df.mean()

Closed         0.822590
Open           0.163989
In Progress    0.013421
dtype: float64

In [26]:
stat_freq_df.std()

Closed         0.162635
Open           0.163459
In Progress    0.016591
dtype: float64

In [27]:
#make difference between none and none set / tbd?
resos = ['Open', 'Fixed', 'Rejected', 'Duplicate']

reso_freq_df = pd.DataFrame(columns=resos)

for s in issue_dict.keys():
    temp = []
    issues = issue_dict[s]
    issues['map_res'] = issues['resolution'].map(resolution_map)
    for r in resos:
        try:
            temp.append(issues.map_res.value_counts()[r]/(len(issues)))
        except:
            temp.append(0)
    reso_freq_df.loc[s] = temp

In [28]:
dup_link_quality = pd.DataFrame(columns=['Repository', 'DuplicateLinks', 'DupLinksWithoutDup', 'DoubleDup'])

for s in SOURCES:
    print(s.upper())
    issues = issue_dict[s]
    links = link_dict[s]
            
    projects = issues[['issue_id', 'map_res']]

    temp = links.merge(projects, left_on = 'issue_id_1', right_on='issue_id')
    temp = temp.merge(projects, left_on = 'issue_id_2', right_on='issue_id', suffixes=('_1', '_2'))
    
    temp['has_dup'] = (temp['map_res_1']=='Duplicate')|(temp['map_res_2']=='Duplicate')
    temp['both_dup'] = (temp['map_res_1']=='Duplicate')&(temp['map_res_2']=='Duplicate')
    
    try:
        dl = len(temp[temp['maptype']=='Duplicate'])
        dlwd = len(temp[(temp['has_dup']==False)&((temp['maptype']=='Duplicate'))])/dl
        dd = len(temp[(temp['both_dup']==True)&((temp['maptype']=='Duplicate'))])/dl
    except:
        dl = 0
        dlwd = 0
        dd = 0
    dup_link_quality.loc[len(dup_link_quality)] = [s, dl, dlwd, dd]
    
#     print(temp[(temp['has_dup']==False)&(temp['maptype']=='Duplicate')][['issues','maptype','map_res_1','map_res_2']][:10])
    print(temp[(temp['has_dup']==False)&(temp['maptype']=='Duplicate')][['issues']][:10].values)

APACHE
[["['ZOOKEEPER-2342', 'ZOOKEEPER-2659']"]
 ["['ZOOKEEPER-1162', 'ZOOKEEPER-4332']"]
 ["['ZOOKEEPER-4235', 'ZOOKEEPER-4367']"]
 ["['ZOOKEEPER-2966', 'ZOOKEEPER-4296']"]
 ["['ZOOKEEPER-2101', 'ZOOKEEPER-3496']"]
 ["['ZOOKEEPER-2593', 'ZOOKEEPER-451']"]
 ["['ZOOKEEPER-1651', 'ZOOKEEPER-3179']"]
 ["['ZOOKEEPER-2407', 'ZOOKEEPER-3059']"]
 ["['ZOOKEEPER-2158', 'ZOOKEEPER-3021']"]
 ["['ZOOKEEPER-1998', 'ZOOKEEPER-2965']"]]
HYPERLEDGER
[["['STL-1672', 'STL-1677']"]
 ["['STL-1115', 'STL-901']"]
 ["['STL-1071', 'STL-1103']"]
 ["['STL-556', 'STL-638']"]
 ["['STL-557', 'STL-638']"]
 ["['STL-242', 'STL-247']"]
 ["['STL-312', 'STL-336']"]
 ["['STL-202', 'STL-69']"]
 ["['STL-105', 'STL-61']"]
 ["['INDY-2170', 'INDY-2171']"]]
INTELDAOS
[["['DAOS-4227', 'DAOS-9153']"]
 ["['DAOS-8930', 'DAOS-8946']"]
 ["['DAOS-6933', 'DAOS-8817']"]
 ["['CART-985', 'DAOS-8618']"]
 ["['DAOS-6957', 'DAOS-6977']"]
 ["['DAOS-8313', 'DAOS-8446']"]
 ["['DAOS-5857', 'DAOS-8168']"]
 ["['DAOS-7202', 'DAOS-7212']"]
 ["['DAO

In [29]:
dup_link_quality

,Repository,DuplicateLinks,DupLinksWithoutDup,DoubleDup
0,Apache,25920,0.176235,0.029514
1,Hyperledger,638,0.429467,0.009404
2,IntelDAOS,251,0.254980,0.047809
3,JFrog,643,0.393468,0.006221
4,Jira,21676,0.168527,0.043781
5,JiraEcosystem,1741,0.171740,0.028719
6,MariaDB,1374,0.224163,0.029840
7,Mindville,17,0.470588,0.058824
8,Mojang,193991,0.003583,0.012645
9,MongoDB,8588,0.079879,0.050186


In [37]:
dup_link_quality.to_csv('properties/quality_dup_link.csv', encoding='utf-8', index=True, sep=',')


print(round(dup_link_quality, 3).set_index('Repository').to_latex())

\begin{tabular}{llrr}
\toprule
{} & DuplicateLinks &  DupLinksWithoutDup &  DoubleDup \\
Repository    &                &                     &            \\
\midrule
Apache        &          25920 &               0.176 &      0.030 \\
Hyperledger   &            638 &               0.429 &      0.009 \\
IntelDAOS     &            251 &               0.255 &      0.048 \\
JFrog         &            643 &               0.393 &      0.006 \\
Jira          &          21676 &               0.169 &      0.044 \\
JiraEcosystem &           1741 &               0.172 &      0.029 \\
MariaDB       &           1374 &               0.224 &      0.030 \\
Mindville     &             17 &               0.471 &      0.059 \\
Mojang        &         193991 &               0.004 &      0.013 \\
MongoDB       &           8588 &               0.080 &      0.050 \\
Qt            &           4243 &               0.202 &      0.033 \\
RedHat        &           5912 &               0.267 &      0.018 \\
Sakai

In [34]:
dup_link_quality[dup_link_quality['Repository']!='SecondLife'].mean()

DuplicateLinks        17928.800000
DupLinksWithoutDup        0.229056
DoubleDup                 0.029583
dtype: float64

In [36]:
dup_link_quality[dup_link_quality['Repository']!='SecondLife'].median()

DuplicateLinks        1745.000000
DupLinksWithoutDup       0.201508
DoubleDup                0.029514
dtype: float64

In [35]:
dup_link_quality[dup_link_quality['Repository']!='SecondLife'].std()

DuplicateLinks        49346.931358
DupLinksWithoutDup        0.123777
DoubleDup                 0.015456
dtype: float64

In [38]:
total_dups = []
no_link = []
no_dup_links = []

for s in SOURCES:
    print(s.upper())
    issues = issue_dict[s]
    temp_df = issues[issues['map_res']=='Duplicate']
    dup_issues = set(temp_df.issue_id.values)
    total_dups.append(len(dup_issues))
    no_link.append(temp_df.no_links.value_counts()[0]/len(dup_issues))
    
    links = un_link_dict[s]
    templ_df = links[(links['maptype']=='Duplicate')|(links['maptype']=='Clone')]
    
    issues_with_duplinks = set(templ_df['issue_id_1'].values).union(set(templ_df['issue_id_2'].values))

    no_dup_links.append(len(dup_issues-issues_with_duplinks)/len(dup_issues))
    
    print(issues[issues['issue_id'].isin(list(dup_issues-issues_with_duplinks)[:10])][['issue_id']].values)
    
dups = {
    'Repository': SOURCES,
    'Total Dup': total_dups,
    'Dups without links': no_link,
    'Dups without dup links': no_dup_links
}

dup_df = pd.DataFrame(dups)

APACHE
[['VCL-199']
 ['SPARK-7170']
 ['SCB-1817']
 ['QPIDJMS-385']
 ['PHOENIX-2172']
 ['NUTCH-1032']
 ['KYLIN-1568']
 ['IMPALA-3010']
 ['HBASE-7286']
 ['HBASE-1679']]
HYPERLEDGER
[['FABP-247']
 ['FABCN-92']
 ['FAB-17278']
 ['FAB-16953']
 ['FAB-14124']
 ['FAB-13597']
 ['FAB-12719']
 ['FAB-9962']
 ['FAB-9145']
 ['FAB-5722']]
INTELDAOS
[['DAOS-9273']
 ['DAOS-8510']
 ['DAOS-8421']
 ['DAOS-6775']
 ['DAOS-6169']
 ['DAOS-6093']
 ['DAOS-4117']
 ['DAOS-4022']
 ['DAOS-2937']
 ['DAOS-2687']]
JFROG
[['RTFACT-25230']
 ['RTFACT-21010']
 ['RTFACT-20481']
 ['RTFACT-19974']
 ['RTFACT-18371']
 ['RTFACT-17747']
 ['RTFACT-14664']
 ['RTFACT-1195']
 ['RTFACT-1090']
 ['BI-462']]
JIRA
[['JRASERVER-621']
 ['CONFSERVER-22184']
 ['CONFSERVER-21318']
 ['CLOUD-3991']
 ['BCLOUD-18486']
 ['BCLOUD-13340']
 ['BCLOUD-7801']
 ['BCLOUD-5401']
 ['BAM-12446']
 ['BAM-1623']]
JIRAECOSYSTEM
[['SPFE-26']
 ['PLUG-808']
 ['PLE-684']
 ['AUI-3501']
 ['AUI-1443']
 ['APL-556']
 ['AMKT-2395']
 ['AMKT-571']
 ['ACJIRA-1747']
 ['ACJIRA-

In [39]:
dup_df.to_csv('properties/quality_dup.csv', encoding='utf-8', index=True, sep=',')
dup_df

,Repository,Total Dup,Dups without links,Dups without dup links
0,Apache,42285,0.340948,0.463663
1,Hyperledger,666,0.238739,0.417417
2,IntelDAOS,570,0.422807,0.601754
3,JFrog,533,0.191370,0.243902
4,Jira,24714,0.174071,0.262442
5,JiraEcosystem,2363,0.275074,0.368176
6,MariaDB,1605,0.167601,0.329595
7,Mindville,24,0.583333,0.583333
8,Mojang,192376,0.007137,0.011472
9,MongoDB,10830,0.118837,0.240628


In [44]:
print(round(dup_df, 3).set_index('Repository').to_latex())

\begin{tabular}{lrrr}
\toprule
{} &  Total Dup &  Dups without links &  Dups without dup links \\
Repository    &            &                     &                         \\
\midrule
Apache        &      42285 &               0.341 &                   0.464 \\
Hyperledger   &        666 &               0.239 &                   0.417 \\
IntelDAOS     &        570 &               0.423 &                   0.602 \\
JFrog         &        533 &               0.191 &                   0.244 \\
Jira          &      24714 &               0.174 &                   0.262 \\
JiraEcosystem &       2363 &               0.275 &                   0.368 \\
MariaDB       &       1605 &               0.168 &                   0.330 \\
Mindville     &         24 &               0.583 &                   0.583 \\
Mojang        &     192376 &               0.007 &                   0.011 \\
MongoDB       &      10830 &               0.119 &                   0.241 \\
Qt            &       9703 &       

In [41]:
dup_df.mean()

Total Dup                 19318.875000
Dups without links            0.331801
Dups without dup links        0.447177
dtype: float64

In [42]:
dup_df.std()

Total Dup                 47497.602084
Dups without links            0.219134
Dups without dup links        0.220209
dtype: float64

In [43]:
dup_df.median()

Total Dup                 2627.500000
Dups without links           0.308011
Dups without dup links       0.440540
dtype: float64